In [2]:
import os
import sys
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import skimage.io as io
import shutil

import ipywidgets as widgets
from IPython.display import display

In [3]:
%load_ext autoreload
%autoreload 2

base_dir = Path("../../pipeline/src").resolve()
sys.path.append(str(base_dir))
sys.path.append(str(base_dir / "utils"))

from utils.parameters_tracking import Parameter_tracking as Track
from utils.widgets import StringListWidget as Slw
from utils.file_handling import FileProcessor as Fp

fp  = Fp()
tk  = Track()

In [ ]:
# start with Tile3, then do Tile 2

In [4]:
import glob
import os
import re
import tifffile as tf
import numpy as np

base_dir = '/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star'   
output_dir = os.path.join(base_dir, 'T2_D_R1_grouped')

os.makedirs(output_dir, exist_ok=True)

In [ ]:
all_files = glob.glob(os.path.join(base_dir, '*.tif'))

In [ ]:
option_t = False # False for tile2, True for tile 3

groups = {}
print(f"Found {len(all_files)} files. Starting grouping...")
for full_path in all_files:
    if option_t:
        match = re.search(r'_t(\d+)_s(\d+)_z(\d+)_ch(\d+)\.tif$', full_path)
    else:
        match = re.search(r'_s(\d+)_z(\d+)_ch(\d+)\.tif$', full_path)
            
    if match:
        if option_t:
            t_val, s_val, z_val, ch_val = match.groups()
            group_key = f't{t_val}_s{s_val}_ch{ch_val}'
        else:
            s_val, z_val, ch_val = match.groups()
            group_key = f's{s_val}_ch{ch_val}' 
        
        if group_key not in groups:
            groups[group_key] = []
        
        groups[group_key].append(full_path)
    else:
        print(f"Warning: File did not match pattern: {os.path.basename(full_path)}")

In [ ]:
for group_key, file_list in groups.items():
    print(f"\nProcessing group: {group_key} ({len(file_list)} files)")
    file_list.sort() 
    image_stack = []
    for file_path in file_list:
        try:
            img = tf.imread(file_path) 
            image_stack.append(img)
        except Exception as e:
            print(f"Error reading {os.path.basename(file_path)}: {e}")
            
    if not image_stack:
        continue

    # 3.3. Concatenate the stack
    # Concatenating along the first axis (axis=0) creates the final T/Z stack
    final_image = np.stack(image_stack, axis=0) 
    
    # 3.4. Define output filename
    # Use a descriptive name based on the first file and the group key
    if option_t:
        base_name = os.path.basename(file_list[0]).split('_t')[0] # Remove tXX, zXX, chXX
    else:
        base_name = os.path.basename(file_list[0]).split('_s')[0] # Remove tXX, zXX, chXX

    print(base_name)
    
    output_filename = f'{base_name}_{group_key}.tif'
    output_path = os.path.join(output_dir, output_filename)
    
    # 3.5. Export the final stacked image
    tf.imwrite(
        output_path, 
        final_image, 
        imagej=True, # Optional: For better compatibility with ImageJ/Fiji
        metadata={'axes': 'ZYX'})
    
    print(f"✅ Exported: {output_filename} with shape {final_image.shape}")

print("\nProcessing complete.")

# convention :

T3_D_R1_grouped :  

t09: empty file

t08: round 10 

t07: round 9   

In [ ]:
def replace_with_dict(match):
    replacement_map = {'_t00': '_r2', '_t01': '_r3', '_t02': '_r4',
                       '_t03': '_r5', '_t04': '_r6', '_t05': '_r7',
                       '_t06': '_r8'}
    key = match.group(0)
    return replacement_map.get(key, key)

PATTERN = r'(_t\d+)'

In [ ]:
# rename the files:
# this is for T3_D_R1

p = Path(output_dir).glob('**/*')
files = [x for x in p if x.is_file()]

for file in files:
    folder_name   = Path(str(file.parent)) 
    new_file_name = Path(file.stem.replace(" ", "").split('_ch')[0] + '_FISH.tif')
    new_filename  = re.sub(PATTERN, replace_with_dict, str(new_file_name))
    new_filename  = re.sub(r'_TileScan[23]', '', new_filename)

    new_filename  = str(folder_name / Path(new_filename))
    try:
        os.rename(str(file), new_filename)
        print(f" Renamed file successfully to: {new_file_name}")
    except Exception as e:
        print(f" An error occurred during rename: {e}")

##  T2_D_R1_grouped :

In [ ]:
def replace_with_dict(match):
    replacement_map = {'_ch00': '_DAPI.tif', '_ch01': '_FISH.tif'}
    key = match.group(0)
    return replacement_map.get(key, key)

PATTERN = r'(_ch\d+)'

In [ ]:
p = Path(output_dir).glob('**/*')
files = [x for x in p if x.is_file()]

for file in files:
    folder_name   = Path(str(file.parent)) 
    new_filename  = Path(file.stem.replace(" ", ""))
    new_filename  = re.sub(r'(_ch\d+)', replace_with_dict, str(new_filename))
    new_filename  = re.sub(r"_s(\d+)", r"_r1_s\1", new_filename)
    new_filename = re.sub(r'_TileScan[23]', '', new_filename)
    new_filename  = str(folder_name / Path(new_filename))
    try:
        os.rename(str(file), new_filename)
        print(f"✅ Renamed file successfully to: {new_filename}")
    except Exception as e:
        print(f" An error occurred during rename: {e}")

In [ ]:
# multiply the files copying them: r1 is copied and renamed to r2, ..., r8
import shutil

p = Path(output_dir).glob('**/*')
files = [x for x in p if x.is_file()]

# create a subselection of DAPI files 
files_sub = [el for el in files if el.stem.split('_')[-1] == "DAPI"]

rs_subs     = ['_r2','_r3','_r4','_r5','_r6','_r7','_r8']
folder_dest = Path('/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped')

for file in files_sub:
    file_name   = file.name
    for rs in rs_subs:
        new_filename_temp = re.sub(r"_r1", rs, file_name)
        file_name2        = str(folder_dest / Path(new_filename_temp))
        shutil.copy2(file, file_name2)
        

In [ ]:

p     = Path(file_name2).parent.glob('**/*')
files = [x for x in p if x.is_file()]


files_sub_DAPI = [str(el) for el in files if el.stem.split('_')[-1] == "DAPI"]
files_sub_DAPI.sort()


files_sub_FISH = [str(el) for el in files if el.stem.split('_')[-1] == "FISH"]
files_sub_FISH.sort()

for el in files_sub_DAPI:
    print('Path("'+el+'"),')



#p = Path(output_dir).glob('**/*')
#files = [x for x in p if x.is_file()]

In [ ]:
output_dir = '/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped'
p     = Path(output_dir).glob('**/*')
files = [x for x in p if x.is_file()]
files.sort()

files_sub_DAPI = [str(el) for el in files if el.stem.split('_')[-1] == "DAPI"]
for el in files_sub_DAPI:
    print('Path("'+el+'"),')


#files_sub_FISH = [str(el) for el in files if el.stem.split('_')[-1] == "FISH"]
#for el in files_sub_FISH:
#    print('Path("'+el+'"),')
